In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect("chinook.db")

In [ ]:
# 1. Customer Purchases Analysis:
# Read customers and invoices tables
customers_df = pd.read_sql("SELECT CustomerId, FirstName, LastName FROM customers", conn)
invoices_df = pd.read_sql("SELECT CustomerId, Total FROM invoices", conn)

# Calculate total amount spent per customer
customer_totals = invoices_df.groupby("CustomerId")["Total"].sum().reset_index()
customer_totals = customer_totals.rename(columns={"Total": "TotalSpent"})

# Merge with customer details
customer_summary = pd.merge(customer_totals, customers_df, on="CustomerId")

# Get top 5 customers
top_5_customers = customer_summary.sort_values(by="TotalSpent", ascending=False).head(5)

# Reorder columns
top_5_customers = top_5_customers[["CustomerId", "FirstName", "LastName", "TotalSpent"]]

print(top_5_customers)

    CustomerId FirstName    LastName  TotalSpent
5            6    Helena        Holý       49.62
25          26   Richard  Cunningham       47.62
56          57      Luis       Rojas       46.62
44          45  Ladislav      Kovács       45.62
45          46      Hugh    O'Reilly       45.62


In [ ]:
# 2. Album vs. Individual Track Purchases:
# Load necessary tables
invoice_lines = pd.read_sql("SELECT InvoiceLineId, InvoiceId, TrackId FROM invoice_items", conn)
invoices = pd.read_sql("SELECT InvoiceId, CustomerId FROM invoices", conn)
tracks = pd.read_sql("SELECT TrackId, AlbumId FROM tracks", conn)
albums = pd.read_sql("SELECT AlbumId FROM albums", conn)

# Merge all tables to get: CustomerId, AlbumId, TrackId
invoice_with_tracks = invoice_lines.merge(invoices, on="InvoiceId")
invoice_with_album = invoice_with_tracks.merge(tracks, on="TrackId")

# Count how many tracks exist per album
album_track_counts = tracks.groupby("AlbumId")["TrackId"].count().reset_index()
album_track_counts = album_track_counts.rename(columns={"TrackId": "TotalAlbumTracks"})

# Count how many tracks each customer purchased per album
customer_album_purchases = invoice_with_album.groupby(["CustomerId", "AlbumId"])["TrackId"].count().reset_index()
customer_album_purchases = customer_album_purchases.rename(columns={"TrackId": "PurchasedTracks"})

# Merge to get total album tracks
customer_album_purchases = customer_album_purchases.merge(album_track_counts, on="AlbumId")

# Determine if the customer bought the full album
customer_album_purchases["FullAlbumPurchased"] = customer_album_purchases["PurchasedTracks"] == customer_album_purchases["TotalAlbumTracks"]

# Group by customer: if any full album purchase, they are album buyers
customer_pref = customer_album_purchases.groupby("CustomerId")["FullAlbumPurchased"].any().reset_index()

# Label preference
customer_pref["Preference"] = customer_pref["FullAlbumPurchased"].apply(lambda x: "Full Albums" if x else "Individual Tracks")

# Summary percentage
summary = customer_pref["Preference"].value_counts(normalize=True) * 100
summary = summary.reset_index()
summary.columns = ["Preference", "Percentage"]

print(summary)

          Preference  Percentage
0  Individual Tracks    64.40678
1        Full Albums    35.59322
